In [20]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
automated_labelling
dataset
labelled_citing_papers
lemos_R_001_sentiment_binary_002_15ep_bert.keras
lemos_R_001_sentiment_predictor_001_12ep_bert.keras
model_0_predictions
model_0_predictions_for_all_42000
model_6_predictions_related_only
plots
R_001_2_label_binary_sentiment_classification_model_training.ipynb
R_001_3_label_multiclass_sentiment_classification_model_training.ipynb
R_001_binary_text_classification_related_not_related.ipynb
R_001_M6_3_class_sentiment_classification.ipynb
R_001_M7_1_binary_classification_related_not_related.ipynb
R_001_multiclass_text_classification.ipynb
R_001_Sentiment_Analysis.ipynb
R_001_wahab.rdp
related_bin


In [4]:
!nvidia-smi

Thu Mar 28 21:18:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatib

# Note: If there are dependency issues, use Google Colab for model training

In [6]:
# -*- coding: utf-8 -*-
"""
R_001_Research_Project__
|
R_001_multiclass_text_classification.ipynb
Created on Thu Jan 18 16:30:03 2024
@author: Rochana Obadage
"""

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_core as keras
import keras_nlp

import pytz
from datetime import datetime
import re
import argparse
import sys

Using TensorFlow backend


In [24]:
# !ls
# !pwd

In [7]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

2024-03-28 21:21:14.656130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load the datasets
```for_training.csv```

```

They contain
    context -- text
    label_scored:{ -2, -1, 0, 0.5, 1 }
    labeln:{"O-NR", "P-NR", "Neutral", "Weak", "Strong"}
```t

In [22]:
df_dataset = pd.read_csv(r"../data/1937_dataset_for_3_label_sentiment.csv")
df_dataset


,context,label_score,label,3_label,target
0,"Recently, several losses [18], [24], [25] cons...",0.0,Neutral,neutral,1
1,The subsequent designs consider multiple negat...,0.0,Neutral,neutral,1
2,"the mainstream methods [1], [5], [6], [18], [6...",0.0,Neutral,neutral,1
3,"Accordingly, several works [24], [25] based on...",0.0,Neutral,neutral,1
4,The proposed method can be seamlessly implante...,0.5,Weak,positive,2
...,...,...,...,...,...
1932,A Jain [125] suggests training a deep network ...,0.0,Neutral,neutral,1
1933,[39]12 2021 No name ICDAR PubTabNet ICDAR 2019...,0.0,Neutral,neutral,1
1934,The source codes of 6 papers were not executab...,-2.0,O-NR,negative,0
1935,The success of DL has marked the revisiting of...,0.0,Neutral,neutral,1


In [25]:
mapping = {'negative':0, 'neutral':1, 'positive':2}

In [26]:
mapping_reverse = dict((value,key) for key,value in mapping.items())
mapping_reverse


{0: 'negative', 1: 'neutral', 2: 'positive'}

In [27]:
# label counts per each category
df_ = df_dataset.groupby('3_label').count()
df_.reset_index()

,3_label,context,label_score,label,target
0,negative,23,23,23,23
1,neutral,1756,1756,1756,1756
2,positive,158,158,158,158


In [28]:
df_dataset_neutral = df_dataset[df_dataset['3_label'] =='neutral'].sample(n=35, replace=False)
df_dataset_positive = df_dataset[df_dataset['3_label'] =='positive'].sample(n=35, replace=False)
df_dataset_negative = df_dataset[df_dataset['3_label'] =='negative']

df_balanced = pd.concat([df_dataset_neutral, df_dataset_positive,df_dataset_negative], ignore_index = True)
df_balanced = df_balanced.sample(frac = 1).reset_index(drop=True)
df_balanced
# df_balanced.to_csv(r'dataset/1937_dataset_for_3_label_sentiment_test_balanced_93.csv',index=False)

,context,label_score,label,3_label,target
0,Following the work of Schreiber et al. (2017) ...,0.5,Weak,positive,2
1,The source codes of 6 papers were not executab...,-2.0,O-NR,negative,0
2,"In contrast, we interpret the information loss...",0.0,Neutral,neutral,1
3,We further observe that empty cells account fo...,0.5,Weak,positive,2
4,Although we tried to train split network with ...,-2.0,O-NR,negative,0
...,...,...,...,...,...
88,"In the same spirit, CrossWalk [34] is based on...",0.0,Neutral,neutral,1
89,Note that LGPMA requires additional\nannotatio...,-1.0,P-NR,negative,0
90,we compared the performance of the four differ...,0.5,Weak,positive,2
91,This reproducibility study focuses on Antoniak...,1.0,Strong,positive,2


In [29]:
df_b = df_balanced.groupby('3_label').count()
df_b.reset_index()

,3_label,context,label_score,label,target
0,negative,23,23,23,23
1,neutral,35,35,35,35
2,positive,35,35,35,35


In [30]:
# lable names
print(set(df_dataset['3_label']))
print(set(df_dataset['target']))

{'negative', 'positive', 'neutral'}
{0, 1, 2}


# Load a DistilBERT model from Keras NLP

In [35]:
BATCH_SIZE = 16
NUM_TRAINING_EXAMPLES = df_balanced.shape[0]
TRAIN_SPLIT = 1
VAL_SPLIT = 0
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 12 #8
AUTO = tf.data.experimental.AUTOTUNE
NUM_TRAINING_EXAMPLES

93

In [33]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=512,
                                                                   )
# preprocessor_4_tweets
# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=3)

# vgg_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
classifier.summary()




100%|██████████| 580/580 [00:00<00:00, 413kB/s]
100%|██████████| 226k/226k [00:00<00:00, 692kB/s]
100%|██████████| 515/515 [00:00<00:00, 505kB/s]
100%|██████████| 253M/253M [00:08<00:00, 32.6MB/s]


Preprocessor: "distil_bert_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)        │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 3)                 │           2,307 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,779 (255.42 MB)

 Trainable params: 66,955,779 (255.42 MB)

 Non-trainable params: 0 (0.00 B)

# Train your own model, fine-tuning BERT

In [36]:
from sklearn.model_selection import KFold
from keras_nlp.models import DistilBertClassifier
from sklearn.metrics import classification_report

import numpy as np

fold_accuracies = []
fold_reports = []
fold_matrices = []

df_train_for_5_fold = df_balanced.copy()

kfold = KFold(n_splits=5, shuffle=True)

for train_index, test_index in kfold.split(df_train_for_5_fold):

    # Get the training and test data for the current fold
    X_train, X_test = df_train_for_5_fold.iloc[train_index], df_train_for_5_fold.iloc[test_index]
    y_train, y_test = X_train['target'], X_test['target']


    classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=3)

    # uncomment below if error
    # classifier.fit(x=X_train, y=y_train, batch_size=2)

    # Re-compile (e.g., with a new learning rate)
    classifier.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(1e-5),
        metrics= ["accuracy"],
    )

    history = classifier.fit(x=X_train['context'],
                            y=y_train,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            )

    # Evaluate the model on the test data
    y_pred = classifier.predict(X_test['context'])


    # # Calculate the accuracy
    accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    print(f"Accuracy: {accuracy}")

    print('\n\n')
    # label_to_score = {'related':1,"not_related":0}
    print('classifiation report : 5-Fold Cross Validation')
    print(classification_report(y_test, np.argmax(y_pred, axis=1),target_names= mapping.keys()))


    # # Calculate the accuracy
    # accuracy = np.mean(y_pred == y_test)
    fold_accuracies.append(accuracy)

    # Generate classification report
    report = classification_report(y_test, np.argmax(y_pred, axis=1),target_names= mapping.keys())
    # report = classification_report(y_test, y_pred, output_dict=True)
    fold_reports.append(report)

    # # Generate confusion matrix
    matrix = confusion_matrix(y_test, np.argmax(y_pred, axis=1))
    fold_matrices.append(matrix)

# Print the average accuracy across all folds
print("Average Accuracy:", np.mean(fold_accuracies))

# Print the classification reports for each fold
for i, report in enumerate(fold_reports):
    print(f"Fold {i+1} Classification Report:")
    print(report)

# Print the confusion matrices for each fold
for i, matrix in enumerate(fold_matrices):
    print(f"Fold {i+1} Confusion Matrix:")
    print(matrix)


Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 116s 13s/step - accuracy: 0.3544 - loss: 1.0987
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 573ms/step - accuracy: 0.3400 - loss: 1.0920
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Accuracy: 0.3157894736842105



classifiation report : Training
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         3
     neutral       0.33      0.11      0.17         9
    positive       0.31      0.71      0.43         7

    accuracy                           0.32        19
   macro avg       0.22      0.28      0.20        19
weighted avg       0.27      0.32      0.24        19



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 113s 12s/step - accuracy: 0.3298 - loss: 1.0955
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 50s 574ms/step - accuracy: 0.4668 - loss: 1.0617
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Accuracy: 0.15789473684210525



classifiation report : Training
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         8
     neutral       0.00      0.00      0.00         8
    positive       0.16      1.00      0.27         3

    accuracy                           0.16        19
   macro avg       0.05      0.33      0.09        19
weighted avg       0.02      0.16      0.04        19



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 128s 13s/step - accuracy: 0.3194 - loss: 1.1035
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 577ms/step - accuracy: 0.4686 - loss: 1.0765
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Accuracy: 0.3684210526315789



classifiation report : Training
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         7
     neutral       0.33      0.17      0.22         6
    positive       0.38      1.00      0.55         6

    accuracy                           0.37        19
   macro avg       0.24      0.39      0.26        19
weighted avg       0.22      0.37      0.24        19



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 139s 16s/step - accuracy: 0.3247 - loss: 1.0998
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 584ms/step - accuracy: 0.4652 - loss: 1.0814


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Accuracy: 0.4444444444444444



classifiation report : Training
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       0.38      1.00      0.55         6
    positive       1.00      0.09      0.17        11

    accuracy                           0.44        18
   macro avg       0.79      0.70      0.57        18
weighted avg       0.79      0.44      0.34        18

Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 118s 13s/step - accuracy: 0.4139 - loss: 1.0900
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 46s 585ms/step - accuracy: 0.4198 - loss: 1.0803


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Accuracy: 0.5555555555555556



classifiation report : Training
              precision    recall  f1-score   support

    negative       1.00      0.25      0.40         4
     neutral       1.00      0.17      0.29         6
    positive       0.50      1.00      0.67         8

    accuracy                           0.56        18
   macro avg       0.83      0.47      0.45        18
weighted avg       0.78      0.56      0.48        18

Average Accuracy: 0.3684210526315789
Fold 1 Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         3
     neutral       0.33      0.11      0.17         9
    positive       0.31      0.71      0.43         7

    accuracy                           0.32        19
   macro avg       0.22      0.28      0.20        19
weighted avg       0.27      0.32      0.24        19

Fold 2 Classification Report:
              precision    recall  f1-sco

In [37]:
saved_model_path = 'lemos_R_001_M6_sentiment_predictor_001_12ep_bert.keras'
classifier.save(saved_model_path)

In [42]:
df_test_1 = pd.read_csv(r'../data/1937_dataset_for_3_label_sentiment_test_balanced_69.csv')
df_test_1

,context,label_score,label,3_label,target
0,With the fast-paced development of digital tra...,0.0,Neutral,neutral,1
1,Although we tried to train split network with ...,-2.0,O-NR,negative,0
2,Note that LGPMA requires additional\nannotatio...,-1.0,P-NR,negative,0
3,"For example, Schreiber et al. (2017) [4]) prop...",0.0,Neutral,neutral,1
4,"Additionally, it is benchmarked with a number ...",0.5,Weak,positive,2
...,...,...,...,...,...
64,The proposed method can be seamlessly implante...,0.5,Weak,positive,2
65,"For comparison, we also visualize the multi-he...",0.5,Weak,positive,2
66,"However, we were unable to obtain reasonable p...",-2.0,O-NR,negative,0
67,We build the TFE as an attention bi-directiona...,0.5,Weak,positive,2


In [44]:
reloaded_model_22 = tf.keras.models.load_model('lemos_R_001_M6_sentiment_predictor_001_12ep_bert.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 209 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [46]:
y_pred = reloaded_model_22.predict(df_test_1['context']) #69 samples

print('\n\n')
# label_to_score = {'related':1,"not_related":0}
print('MODEL 0')
print(classification_report(df_test_1['target'], np.argmax(y_pred, axis=1),target_names= mapping.keys()))

3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step



MODEL 0
              precision    recall  f1-score   support

    negative       0.82      1.00      0.90        23
     neutral       0.75      0.65      0.70        23
    positive       0.76      0.70      0.73        23

    accuracy                           0.78        69
   macro avg       0.78      0.78      0.78        69
weighted avg       0.78      0.78      0.78        69



# Predictions using the model M6

In [50]:
import glob
import os


FOLDER_NAME = r'../data/M6_predictions_for_all_41244/'

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)


label_file_list = [i for i in glob.glob("../data/Citing_Paper_contexts_csv/*.csv")]
count = 0

for file_ in label_file_list:
    df_ = pd.read_csv(file_)
    print(len(df_))

    predictions_df = df_.copy()
    # predictions_df = predictions_df[['text','target_predict','target_predict_label']]#.reset_index(drop=True)
    predictions_df = predictions_df[['text']]#.reset_index(drop=True)

    if len(predictions_df) == 0:
        continue



    predictions_df['target_M6_predict'] = 0
    x_test_series = predictions_df['text']


    predictions = reloaded_model_22.predict(x_test_series)
    predictions_df["target_M6_predict"] = np.argmax(predictions, axis=1)

    mapping_reverse = {0: 'negative', 1: 'neutral', 2: 'positive'}
    predictions_df['target_predict_M6_label'] = predictions_df["target_M6_predict"].map(mapping_reverse)
    full_filename = os.path.join(FOLDER_NAME, os.path.basename(file_))
    print(full_filename)

    print("\n\n\n")
    # print(predictions_df)

    predictions_df.to_csv(full_filename,index=False)
    print(full_filename + ' saved')
    print(count)
    count+=1

226
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 461ms/step
../data/M6_predictions_for_all_41244/RS_006_MLRC_2022_06.csv




../data/M6_predictions_for_all_41244/RS_006_MLRC_2022_06.csv saved
0
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
../data/M6_predictions_for_all_41244/RS_014_MLRC_2022_14.csv




../data/M6_predictions_for_all_41244/RS_014_MLRC_2022_14.csv saved
1
52
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 499ms/step
../data/M6_predictions_for_all_41244/RS_032_MLRC_2022_32.csv




../data/M6_predictions_for_all_41244/RS_032_MLRC_2022_32.csv saved
2
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
../data/M6_predictions_for_all_41244/RS_023_MLRC_2022_23.csv




../data/M6_predictions_for_all_41244/RS_023_MLRC_2022_23.csv saved
3
55
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 437ms/step
../data/M6_predictions_for_all_41244/RS_036_MLRC_2022_36.csv




../data/M6_predictions_for_all_41244/RS_036_MLRC_2022_36.csv saved
4
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
../data/M6_predictions_for_all_41244/RS_030_MLRC_2022_30.csv




../data/M6_p